In [1]:
from lohrasb.best_estimator import BaseModel
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import KFold,train_test_split
import pandas as pd
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import mean_absolute_error,r2_score
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show


2023-07-03 02:55:44,017 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-03 02:55:44,022 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-03 02:55:44,025 :: matplotlib :: interactive is False
2023-07-03 02:55:44,026 :: matplotlib :: platform is darwin
2023-07-03 02:55:44,041 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 02:55:44,043 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 02:55:44,044 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-03 02:55:44,046 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-03 02:55:44,051 :: graphviz._tools :: deprecate positio

#### Example 2 :Computer Hardware Data Set (a regression problem)
  
https://archive.ics.uci.edu/ml/datasets/Computer+Hardware

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data"
# column names
col_names=[
    "vendor name",
    "Model Name",
    "MYCT",
    "MMIN",
    "MMAX",
    "CACH",
    "CHMIN",
    "CHMAX",
    "PRP"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data

,vendor name,Model Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP
adviser,32/60,125,256,6000,256,16,128,198,199
amdahl,470v/7,29,8000,32000,32,8,32,269,253
amdahl,470v/7a,29,8000,32000,32,8,32,220,253
amdahl,470v/7b,29,8000,32000,32,8,32,172,253
amdahl,470v/7c,29,8000,16000,32,8,16,132,132
...,...,...,...,...,...,...,...,...,...
sperry,80/8,124,1000,8000,0,1,8,42,37
sperry,90/80-model-3,98,1000,8000,32,2,8,46,50
sratus,32,125,2000,8000,0,2,14,52,41
wang,vs-100,480,512,8000,32,0,0,67,47


#### Train test split

In [3]:
X = data.loc[:, data.columns != "PRP"]
y = data.loc[:, data.columns == "PRP"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Find feature types for later use

In [4]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


####  Define estimator and set its arguments  

In [5]:
estimator = ExplainableBoostingRegressor()
estimator_params= {
                    "learning_rate":[0.01,0.1], 
                    "validation_size":[0.15,0.20],
                    "early_stopping_rounds":[50,200], 
                    "early_stopping_tolerance":[0.00001,0.001], 
                    "max_rounds":[5000,20000], 
                    "min_samples_leaf":[2,5], 
                    "max_leaves":[2,5]  
                      }

kwargs = {  # params for fit method  
            'fit_optuna_kwargs' :{
            'sample_weight':None,
            },
            # params for OptunaSearch
            'main_optuna_kwargs' : {
            'estimator':estimator,
            'estimator_params':estimator_params,
            'refit':True,
            'measure_of_accuracy':"mean_absolute_error(y_true, y_pred, multioutput='uniform_average')",

            },
            'train_test_split_kwargs':{
                'test_size':.3,
                            
            },
            'study_search_kwargs':{
                'storage':None,
                'sampler':TPESampler(),
                'pruner':HyperbandPruner(),
                'study_name':"example of optuna optimizer",
                'direction':"maximize",
                'load_if_exists':False,
            },
            'optimize_kwargs':{
                # optuna optimization params
                'n_trials':20,
                'timeout':600,
                'catch':(),
                'callbacks':None,
                'gc_after_trial':False,
                'show_progress_bar':False,
            }
}



In [6]:
obj = BaseModel().optimize_by_optuna(
    kwargs=kwargs
        )

#### Build sklearn pipeline

In [7]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # regression model 
            ('obj', obj),


 ])



#### Run Pipeline

In [8]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


[I 2023-07-03 02:55:45,049] A new study created in memory with name: example of optuna optimizer


2023-07-03 02:55:45,053 :: interpret.utils._native :: EBM lib loading.
2023-07-03 02:55:45,054 :: interpret.utils._native :: Loading native on Darwin | debug = False
2023-07-03 02:55:45,097 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:48,555 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,560 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,564 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,567 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,569 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,572 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,574 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,577 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:48,580 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:48,584 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:55:49,113 :: interpret.glassbox.ebm.bin 

[I 2023-07-03 02:55:49,127] Trial 0 finished with value: 15.28016061904285 and parameters: {'learning_rate': 0.08442422226849397, 'validation_size': 0.18890402842276124, 'early_stopping_rounds': 150, 'early_stopping_tolerance': 0.0006031334270084262, 'max_rounds': 15702, 'min_samples_leaf': 2, 'max_leaves': 4}. Best is trial 0 with value: 15.28016061904285.


2023-07-03 02:55:49,139 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:49,205 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,207 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,213 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,217 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,219 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,221 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,223 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,225 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,228 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:49,232 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:55:49,377 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:49,387 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 02:55:49,390] Trial 1 finished with value: 15.87785888056342 and parameters: {'learning_rate': 0.09187904123448502, 'validation_size': 0.1982819193337451, 'early_stopping_rounds': 78, 'early_stopping_tolerance': 0.0003846010921962016, 'max_rounds': 14889, 'min_samples_leaf': 3, 'max_leaves': 5}. Best is trial 1 with value: 15.87785888056342.


...

2023-07-03 02:55:57,427 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:57,549 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,550 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,552 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,555 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,558 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,560 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,563 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,566 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:57,568 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:57,571 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:55:57,995 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,005 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 02:55:58,008] Trial 18 finished with value: 15.698614655927363 and parameters: {'learning_rate': 0.05191375995802262, 'validation_size': 0.1751006734398278, 'early_stopping_rounds': 199, 'early_stopping_tolerance': 0.0004874140803316499, 'max_rounds': 13732, 'min_samples_leaf': 5, 'max_leaves': 3}. Best is trial 2 with value: 16.995655798459644.


2023-07-03 02:55:58,065 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:58,246 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,248 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,250 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,253 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,255 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,257 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,259 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,261 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:58,263 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:55:58,266 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:55:59,058 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:55:59,067 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 02:55:59,071] Trial 19 finished with value: 12.61626762327645 and parameters: {'learning_rate': 0.0742443457806828, 'validation_size': 0.18420476974063638, 'early_stopping_rounds': 160, 'early_stopping_tolerance': 0.0006741898133747349, 'max_rounds': 7258, 'min_samples_leaf': 4, 'max_leaves': 2}. Best is trial 2 with value: 16.995655798459644.


2023-07-03 02:55:59,072 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 02:55:59,072 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 02:55:59,087 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:00,316 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,318 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,321 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,323 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,325 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,327 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,329 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,332 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:00,338 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56

#### Check performance of the pipeline

In [9]:
print('r2 score : ')
print(r2_score(y_test,y_pred))


r2 score : 
0.5503945238854855


#### Part 2:  Use BestModel as a standalone estimator 

In [10]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Transform features to make them ready for model input

In [11]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [12]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [13]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

[I 2023-07-03 02:56:00,765] A new study created in memory with name: example of optuna optimizer


2023-07-03 02:56:00,777 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:01,359 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,362 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,363 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,366 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,369 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,371 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,373 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,375 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:01,378 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:01,381 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:56:03,511 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,523 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 02:56:03,529] Trial 0 finished with value: 11.822874873776374 and parameters: {'learning_rate': 0.014962451796605002, 'validation_size': 0.16245501695446754, 'early_stopping_rounds': 86, 'early_stopping_tolerance': 0.00039942659093249487, 'max_rounds': 10597, 'min_samples_leaf': 2, 'max_leaves': 5}. Best is trial 0 with value: 11.822874873776374.


2023-07-03 02:56:03,543 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:03,828 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,830 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,832 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,834 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,837 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,840 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,841 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,844 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:03,846 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:03,849 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:56:04,374 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:04,383 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 02:56:04,387] Trial 1 finished with value: 12.132147968812092 and parameters: {'learning_rate': 0.053511462599148234, 'validation_size': 0.16198141168617516, 'early_stopping_rounds': 64, 'early_stopping_tolerance': 0.0003788489842755574, 'max_rounds': 16401, 'min_samples_leaf': 2, 'max_leaves': 4}. Best is trial 1 with value: 12.132147968812092.


...

2023-07-03 02:56:21,765 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:21,885 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,887 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,890 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,892 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,893 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,895 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,897 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,899 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:21,901 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:21,904 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:56:22,046 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,055 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 02:56:22,058] Trial 18 finished with value: 12.9140692377137 and parameters: {'learning_rate': 0.07523395236314588, 'validation_size': 0.15702857187364452, 'early_stopping_rounds': 87, 'early_stopping_tolerance': 0.0004886705615876085, 'max_rounds': 17440, 'min_samples_leaf': 5, 'max_leaves': 3}. Best is trial 15 with value: 13.747992725666455.


2023-07-03 02:56:22,104 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:22,288 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,290 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,292 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,295 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,296 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,298 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,300 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,302 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,305 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:22,309 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 02:56:22,463 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:22,472 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 02:56:22,475] Trial 19 finished with value: 11.056757570733762 and parameters: {'learning_rate': 0.06233583214233987, 'validation_size': 0.15716162724245503, 'early_stopping_rounds': 119, 'early_stopping_tolerance': 0.0009977902394311248, 'max_rounds': 19971, 'min_samples_leaf': 4, 'max_leaves': 4}. Best is trial 15 with value: 13.747992725666455.


2023-07-03 02:56:22,475 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 02:56:22,475 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 02:56:22,485 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56:23,368 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,370 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,373 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,375 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,377 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,379 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,381 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,384 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 02:56:23,387 :: interpret.utils._binning :: Creating native dataset
2023-07-03 02:56

#### Check performance of the model

In [14]:
print('r2 score : ')
print(r2_score(y_test,y_pred))
print('mean_absolute_error : ')
print(mean_absolute_error(y_test,y_pred))


r2 score : 
0.5644799870799552
mean_absolute_error : 
43.10811650673652


In [15]:
ebm=obj.get_best_estimator()
print(ebm)

ExplainableBoostingRegressor(early_stopping_rounds=106,
                             early_stopping_tolerance=0.0007106660286424488,
                             learning_rate=0.07082906383268768,
                             max_rounds=17841, min_samples_leaf=4,
                             validation_size=0.1571077038156805)


In [16]:
obj.best_estimator

ExplainableBoostingRegressor(early_stopping_rounds=106,
                             early_stopping_tolerance=0.0007106660286424488,
                             learning_rate=0.07082906383268768,
                             max_rounds=17841, min_samples_leaf=4,
                             validation_size=0.1571077038156805)

#### Get fitted randomized search object and its attributes

In [18]:
OptunaObj = obj.get_optimized_object()
OptunaObj

FrozenTrial(number=15, state=TrialState.COMPLETE, values=[13.747992725666455], datetime_start=datetime.datetime(2023, 7, 3, 2, 56, 19, 720698), datetime_complete=datetime.datetime(2023, 7, 3, 2, 56, 20, 83106), params={'learning_rate': 0.07082906383268768, 'validation_size': 0.1571077038156805, 'early_stopping_rounds': 106, 'early_stopping_tolerance': 0.0007106660286424488, 'max_rounds': 17841, 'min_samples_leaf': 4, 'max_leaves': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'validation_size': FloatDistribution(high=0.2, log=False, low=0.15, step=None), 'early_stopping_rounds': IntDistribution(high=200, log=False, low=50, step=1), 'early_stopping_tolerance': FloatDistribution(high=0.001, log=False, low=1e-05, step=None), 'max_rounds': IntDistribution(high=20000, log=False, low=5000, step=1), 'min_samples_leaf': IntDistribution(high=5, log=False, low=2, step=1), 'max_leaves': IntD

#### This visualizations provided is for global explanations.



In [ ]:
ebm_global = ebm.explain_global()
show(ebm_global)


2023-07-03 02:56:40,033 :: interpret.provider.visualize :: Detected non-cloud environment.
2023-07-03 02:56:40,957 :: interpret.visual.udash :: Generating mini dash
2023-07-03 02:56:40,970 :: interpret.visual.udash :: Generated mini dash


<!-- http://127.0.0.1:7001/5215382416/ -->

2023-07-03 02:56:41,555 :: matplotlib :: CACHEDIR=/Users/hjavedani/.matplotlib
2023-07-03 02:56:41,563 :: matplotlib.font_manager :: Using fontManager instance from /Users/hjavedani/.matplotlib/fontlist-v330.json


#### This visualizations provided is for local explanations.


In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5])
show(ebm_local)
